In [1]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report, multilabel_confusion_matrix
import tensorflow as tf
import datetime, os
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Flatten, concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.constraints import MaxNorm
from sklearn.model_selection import GridSearchCV
from splitrepeat import splitrepeat_cv

features=['moca_digits','moca_letters','moca_serial7','MMSPELL_early','MMSPELL_late','ADAS_Q1','moca_orient','ADAS_Q7','ADAS_Q8','ADAS_Q9','MMDATE',
       'MMYEAR', 'MMMONTH', 'MMDAY', 'MMSEASON', 'MMHOSPIT', 'MMFLOOR','MMCITY', 'MMAREA', 'MMSTATE','nbspan_forward',
       'faq7','faq8','moca_recall','ADAS_Q4','MMBALLDL', 'MMFLAGDL', 'MMTREEDL', 'LDELTOTAL', 'AVRECALL', 'AVDEL30MIN', 'AVDELTOT', 'AVDELERR2',
       'faq9','moca_clock','MMDRAW','CLOCKCIRC', 'CLOCKSYM','CLOCKNUM', 'CLOCKHAND', 'CLOCKTIME', 'COPYCIRC', 'COPYSYM',
       'COPYNUM', 'COPYHAND', 'COPYTIME', 'tmab_time', 'TRAAERRCOM','TRAAERROM', 'TRABERRCOM', 'TRABERROM','moca_naming','moca_repeat','moca_fluency',
       'moca_similarities','ADAS_Q2', 'ADAS_Q5','ADAS_Q10','ADAS_Q11','ADAS_Q12','MMBALL', 'MMFLAG', 'MMTREE',
       'MMWATCH', 'MMPENCIL','MMREPEAT','MMREAD', 'MMWRITE', 'CATANIMSC', 'CATVEGESC', 'moca_visuo_exec','ADAS_Q3','ADAS_Q6','ADAS_Q13','MMHAND', 'MMFOLD',
       'MMONFLR','nbspan_backward','faq1','faq2','faq3', 'faq4','faq5','faq6','faq10','PXGENAPP', 'PXHEADEY', 'PXNECK', 'PXCHEST',
       'PXHEART', 'PXABDOM', 'PXEXTREM', 'PXPERIPH', 'PXSKIN', 'PXMUSCUL', 'e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt',
       'e_divatt_pt','e_memory_cg', 'e_lang_cg', 'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg']


categorical_features = np.in1d(features, ['MMONFLR', 'faq10', 'faq6', 'COPYSYM', 'COPYNUM', 'faq5', 'MMTREE', 'COPYHAND', 'moca_clock', 'MMSEASON', 'moca_letters', 
                                          'MMBALL', 'faq9', 'MMFLOOR', 'MMDRAW', 'MMMONTH', 'PXGENAPP', 'MMWATCH', 'CLOCKCIRC', 'faq8', 'MMHOSPIT', 'moca_naming', 
                                          'PXEXTREM', 'CLOCKNUM', 'PXMUSCUL', 'faq1', 'MMTREEDL', 'CLOCKTIME', 'PXABDOM', 'MMFLAG', 'COPYCIRC', 'MMAREA', 'faq3', 
                                          'moca_digits', 'CLOCKHAND', 'MMREAD', 'MMYEAR', 'MMREPEAT', 'moca_visuo_exec','MMHAND', 'MMBALLDL', 'PXCHEST', 'MMDATE', 
                                          'MMFLAGDL',  'MMSTATE','moca_repeat', 'MMFOLD', 'MMPENCIL', 'MMDAY',  'faq2', 'PXHEART', 'CLOCKSYM', 'faq4', 'moca_serial7', 
                                          'faq7', 'MMCITY', 'PXHEADEY', 'COPYTIME', 'PXPERIPH', 'PXSKIN', 'moca_fluency', 'moca_similarities', 'PXNECK', 'MMWRITE',
                                          'e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt', 'e_divatt_pt','e_memory_cg', 'e_lang_cg', 
                                          'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg'])


data = pd.read_csv('../data/processed/data_adni.csv')

Using TensorFlow backend.


# CDR - detect DX

In [2]:
features = ['CDSOURCE', 'CDVERSION', 'CDMEMORY',
       'CDORIENT', 'CDJUDGE', 'CDCOMMUN', 'CDHOME', 'CDCARE']

X = data[features]
y = data['DX']
y.replace({2:1},inplace=True)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

X, X_test, y, y_test = train_test_split(X, y, train_size=0.25, random_state=33433, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=33433, stratify=y)    
rf = RandomForestClassifier(n_estimators=100, max_features=.5, random_state=0)

# Search parameters
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['sqrt', 'log2', .3, .5, .7, .9]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

search_params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf_search_rand = RandomizedSearchCV(rf, search_params, n_iter=400,cv=3,verbose=2,random_state=33433, n_jobs=-1)
rf_search_rand.fit(X_train, y_train)
rf_search_rand.best_params_

Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   41.7s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:  4.8min finished


{'n_estimators': 1200,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'log2',
 'max_depth': 60,
 'bootstrap': True}

In [13]:
search_params = {'n_estimators': [1100, 1200, 1300],
               'max_features': [.2, 'sqrt', 'log2'],
               'max_depth': [50, 60, 70],
               'min_samples_split': [4, 5, 6],
               'min_samples_leaf': [1, 2, 3],
               'bootstrap': [False, True]}

rf_search_grid = GridSearchCV(rf, search_params, 
                          cv = 3, n_jobs = -1, verbose = 2)
rf_search_grid.fit(X_train, y_train)
rf_search_grid.best_params_

Fitting 3 folds for each of 486 candidates, totalling 1458 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   33.9s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 1458 out of 1458 | elapsed:  6.6min finished


{'bootstrap': False,
 'max_depth': 50,
 'max_features': 0.2,
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 1100}

## best = RandomForestClassifier(n_estimators=1100, max_features=.2, max_depth = 50, min_samples_split = 4, min_samples_leaf = 1, bootstrap=False, random_state=33433)
df = splitrepeat_cv(X,y,best,splits=list(int(x)*42+42 for x in range(20)),repeats=list(int(x)*42+42 for x in range(20)))
display(df.describe())

### Save outputs ###
q = 'CExam_2class_CDR_400'

df['set'] = q
df.to_csv(('../models/outputs/' + q + '.csv'), index=False)

# MMSE - detect CDR

In [5]:
features = ['MMDATE', 'MMYEAR', 'MMMONTH',
       'MMDAY', 'MMSEASON', 'MMHOSPIT', 'MMFLOOR', 'MMCITY', 'MMAREA',
       'MMSTATE', 'MMBALL', 'MMFLAG', 'MMTREE', 'MMBALLDL', 'MMFLAGDL',
       'MMTREEDL', 'MMWATCH', 'MMPENCIL', 'MMREPEAT', 'MMHAND', 'MMFOLD',
       'MMONFLR', 'MMREAD', 'MMWRITE', 'MMDRAW','MMSPELL_early', 'MMSPELL_late']

X = data[features]
y = data['CDGLOBAL']
y.replace({2:1},inplace=True)

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

X, X_test, y, y_test = train_test_split(X, y, train_size=0.25, random_state=33433, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=33433, stratify=y)    
rf = RandomForestClassifier(n_estimators=100, max_features=.5, random_state=0)

# Search parameters
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['sqrt', 'log2', .3, .5, .7, .9]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

search_params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf_search_rand = RandomizedSearchCV(rf, search_params, n_iter=400,cv=3,verbose=2,random_state=33433, n_jobs=-1)
rf_search_rand.fit(X_train, y_train)
rf_search_rand.best_params_

Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   46.9s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:  5.7min finished


{'n_estimators': 400,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 40,
 'bootstrap': False}

In [17]:
search_params = {'n_estimators': [300, 400, 500],
               'max_features': [.2, 'sqrt', 'log2'],
               'max_depth': [30, 40, 50],
               'min_samples_split': [8, 9, 10],
               'min_samples_leaf': [3, 4, 5],
               'bootstrap': [False, True]}

rf_search_grid = GridSearchCV(rf, search_params, 
                          cv = 3, n_jobs = -1, verbose = 2)
rf_search_grid.fit(X_train, y_train)
rf_search_grid.best_params_

Fitting 3 folds for each of 486 candidates, totalling 1458 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   31.0s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:   54.9s
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 1458 out of 1458 | elapsed:  2.4min finished


{'bootstrap': False,
 'max_depth': 30,
 'max_features': 0.2,
 'min_samples_leaf': 4,
 'min_samples_split': 9,
 'n_estimators': 300}

In [6]:
best = RandomForestClassifier(n_estimators=300, max_features=.2, max_depth = 30, min_samples_split = 9, min_samples_leaf = 4, bootstrap=False, random_state=33433)
df = splitrepeat_cv(X,y,best,splits=list(int(x)*42+42 for x in range(20)),repeats=list(int(x)*42+42 for x in range(20)), avg_strategy='weighted', initial_split_seed=33433, initial_split_ratio=.25)
display(df.describe())

### Save outputs ###
q = 'CExam_2class_MMSE_400'

df['set'] = q
df.to_csv(('../models/outputs/' + q + '.csv'), index=False)

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,0.724845,0.690515,0.705578,0.707680,0.690515,0.724845,0.705578,0.707680,0.690515,0.724845,0.705578,0.707680
std,0.042363,0.035503,0.022508,0.022897,0.035503,0.042363,0.022508,0.022897,0.035503,0.042363,0.022508,0.022897
min,0.635659,0.624242,0.656463,0.656730,0.624242,0.635659,0.656463,0.656730,0.624242,0.635659,0.656463,0.656730
25%,0.682171,0.654545,0.687075,0.690275,0.654545,0.682171,0.687075,0.690275,0.654545,0.682171,0.687075,0.690275
50%,0.732558,0.690909,0.705782,0.704299,0.690909,0.732558,0.705782,0.704299,0.690909,0.732558,0.705782,0.704299
75%,0.751938,0.727273,0.724490,0.727484,0.727273,0.751938,0.724490,0.727484,0.727273,0.751938,0.724490,0.727484
max,0.798450,0.745455,0.744898,0.749049,0.745455,0.798450,0.744898,0.749049,0.745455,0.798450,0.744898,0.749049


# MOCA - detect CDR

In [7]:
features = ['moca_visuo_exec', 'moca_clock',
       'moca_naming', 'moca_digits', 'moca_letters', 'moca_serial7',
       'moca_repeat', 'moca_fluency', 'moca_similarities', 'moca_recall',
       'moca_orient']

X = data[features]
y = data['CDGLOBAL']
y.replace({2:1},inplace=True)

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

X, X_test, y, y_test = train_test_split(X, y, train_size=0.25, random_state=33433, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=33433, stratify=y)    
rf = RandomForestClassifier(n_estimators=100, max_features=.5, random_state=0)

# Search parameters
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['sqrt', 'log2', .3, .5, .7, .9]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

search_params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf_search_rand = RandomizedSearchCV(rf, search_params, n_iter=400,cv=3,verbose=2,random_state=33433, n_jobs=-1)
rf_search_rand.fit(X_train, y_train)
rf_search_rand.best_params_

Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   45.3s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:  5.5min finished


{'n_estimators': 800,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'log2',
 'max_depth': 50,
 'bootstrap': True}

In [21]:
search_params = {'n_estimators': [700, 800, 900],
               'max_features': [.2, 'sqrt', 'log2'],
               'max_depth': [40, 50, 60],
               'min_samples_split': [8, 9, 10],
               'min_samples_leaf': [3, 4, 5],
               'bootstrap': [False, True]}

rf_search_grid = GridSearchCV(rf, search_params, 
                          cv = 3, n_jobs = -1, verbose = 2)
rf_search_grid.fit(X_train, y_train)
rf_search_grid.best_params_

Fitting 3 folds for each of 486 candidates, totalling 1458 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   25.0s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   59.5s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 1458 out of 1458 | elapsed:  4.7min finished


{'bootstrap': True,
 'max_depth': 40,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 800}

In [8]:
best = RandomForestClassifier(n_estimators=800, max_features='sqrt', max_depth = 40, min_samples_split = 10, min_samples_leaf = 4, bootstrap=True, random_state=33433)
df = splitrepeat_cv(X,y,best,splits=list(int(x)*42+42 for x in range(20)),repeats=list(int(x)*42+42 for x in range(20)), avg_strategy='weighted', initial_split_seed=33433, initial_split_ratio=.25)
display(df.describe())

### Save outputs ###
q = 'CExam_2class_MOCA_400'

df['set'] = q
df.to_csv(('../models/outputs/' + q + '.csv'), index=False)

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,0.692229,0.698470,0.695731,0.695349,0.698470,0.692229,0.695731,0.695349,0.698470,0.692229,0.695731,0.695349
std,0.042522,0.034459,0.024949,0.025450,0.034459,0.042522,0.024949,0.025450,0.034459,0.042522,0.024949,0.025450
min,0.596899,0.618182,0.653061,0.649471,0.618182,0.596899,0.653061,0.649471,0.618182,0.596899,0.653061,0.649471
25%,0.666667,0.672727,0.680272,0.678700,0.672727,0.666667,0.680272,0.678700,0.672727,0.666667,0.680272,0.678700
50%,0.689922,0.696970,0.687075,0.689641,0.696970,0.689922,0.687075,0.689641,0.696970,0.689922,0.687075,0.689641
75%,0.715116,0.727273,0.714286,0.711628,0.727273,0.715116,0.714286,0.711628,0.727273,0.715116,0.714286,0.711628
max,0.782946,0.793939,0.748299,0.747357,0.793939,0.782946,0.748299,0.747357,0.793939,0.782946,0.748299,0.747357


# ADAS - detect CDR

In [9]:
features = ['ADAS_Q1', 'ADAS_Q2', 'ADAS_Q3',
       'ADAS_Q4', 'ADAS_Q5', 'ADAS_Q6', 'ADAS_Q7', 'ADAS_Q8', 'ADAS_Q9',
       'ADAS_Q10', 'ADAS_Q11', 'ADAS_Q12', 'ADAS_Q13']

X = data[features]
y = data['CDGLOBAL']
y.replace({2:1},inplace=True)

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

X, X_test, y, y_test = train_test_split(X, y, train_size=0.25, random_state=33433, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=33433, stratify=y)    
rf = RandomForestClassifier(n_estimators=100, max_features=.5, random_state=0)

# Search parameters
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['sqrt', 'log2', .3, .5, .7, .9]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

search_params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf_search_rand = RandomizedSearchCV(rf, search_params, n_iter=400,cv=3,verbose=2,random_state=33433, n_jobs=-1)
rf_search_rand.fit(X_train, y_train)
rf_search_rand.best_params_

Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   45.0s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:  5.6min finished


{'n_estimators': 800,
 'min_samples_split': 5,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 80,
 'bootstrap': True}

In [25]:
search_params = {'n_estimators': [700, 800, 900],
               'max_features': [.2, 'sqrt', 'log2'],
               'max_depth': [70, 80, 90],
               'min_samples_split': [4, 5, 6],
               'min_samples_leaf': [3, 4, 5],
               'bootstrap': [False, True]}

rf_search_grid = GridSearchCV(rf, search_params, 
                          cv = 3, n_jobs = -1, verbose = 2)
rf_search_grid.fit(X_train, y_train)
rf_search_grid.best_params_

Fitting 3 folds for each of 486 candidates, totalling 1458 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   25.1s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   59.9s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 1458 out of 1458 | elapsed:  4.7min finished


{'bootstrap': True,
 'max_depth': 70,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 4,
 'n_estimators': 800}

In [10]:
best = RandomForestClassifier(n_estimators=800, max_features='sqrt', max_depth = 70, min_samples_split = 4, min_samples_leaf = 4, bootstrap=True, random_state=33433)
df = splitrepeat_cv(X,y,best,splits=list(int(x)*42+42 for x in range(20)),repeats=list(int(x)*42+42 for x in range(20)), avg_strategy='weighted', initial_split_seed=33433, initial_split_ratio=.25)
display(df.describe())

### Save outputs ###
q = 'CExam_2class_ADAS_400'

df['set'] = q
df.to_csv(('../models/outputs/' + q + '.csv'), index=False)

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,0.728857,0.756000,0.744090,0.742428,0.756000,0.728857,0.744090,0.742428,0.756000,0.728857,0.744090,0.742428
std,0.044487,0.032028,0.023851,0.024777,0.032028,0.044487,0.023851,0.024777,0.032028,0.044487,0.023851,0.024777
min,0.643411,0.690909,0.700680,0.694433,0.690909,0.643411,0.700680,0.694433,0.690909,0.643411,0.700680,0.694433
25%,0.697674,0.733333,0.727891,0.722322,0.733333,0.697674,0.727891,0.722322,0.733333,0.697674,0.727891,0.722322
50%,0.720930,0.745455,0.748299,0.744644,0.745455,0.720930,0.748299,0.744644,0.745455,0.720930,0.748299,0.744644
75%,0.759690,0.781818,0.761905,0.757435,0.781818,0.759690,0.761905,0.757435,0.781818,0.759690,0.761905,0.757435
max,0.852713,0.836364,0.806122,0.810359,0.836364,0.852713,0.806122,0.810359,0.836364,0.852713,0.806122,0.810359


# EverydayCognition - detect CDR

In [11]:
features = ['e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt', 
            'e_divatt_pt','e_memory_cg', 'e_lang_cg', 
           'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg']


X = data[features]
y = data['CDGLOBAL']
y.replace({2:1},inplace=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

X, X_test, y, y_test = train_test_split(X, y, train_size=0.25, random_state=33433, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=33433, stratify=y)    
rf = RandomForestClassifier(n_estimators=100, max_features=.5, random_state=0)

# Search parameters
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['sqrt', 'log2', .3, .5, .7, .9]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

search_params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf_search_rand = RandomizedSearchCV(rf, search_params, n_iter=400,cv=3,verbose=2,random_state=33433, n_jobs=-1)
rf_search_rand.fit(X_train, y_train)
rf_search_rand.best_params_

In [29]:
search_params = {'n_estimators': [700, 800, 900],
               'max_features': [.2, 'sqrt', 'log2'],
               'max_depth': [70, 80, 90],
               'min_samples_split': [4, 5, 6],
               'min_samples_leaf': [3, 4, 5],
               'bootstrap': [False, True]}

rf_search_grid = GridSearchCV(rf, search_params, 
                          cv = 3, n_jobs = -1, verbose = 2)
rf_search_grid.fit(X_train, y_train)
rf_search_grid.best_params_

Fitting 3 folds for each of 486 candidates, totalling 1458 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   25.1s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 1458 out of 1458 | elapsed:  4.7min finished


{'bootstrap': True,
 'max_depth': 70,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 4,
 'n_estimators': 800}

In [12]:
best = RandomForestClassifier(n_estimators=800, max_features='sqrt', max_depth = 70, min_samples_split = 4, min_samples_leaf = 4, bootstrap=True, random_state=33433)
df = splitrepeat_cv(X,y,best,splits=list(int(x)*42+42 for x in range(20)),repeats=list(int(x)*42+42 for x in range(20)), avg_strategy='weighted', initial_split_seed=33433, initial_split_ratio=.25)
display(df.describe())

### Save outputs ###
q = 'CExam_2class_ECog_400'

df['set'] = q
df.to_csv(('../models/outputs/' + q + '.csv'), index=False)

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,0.758101,0.841530,0.804923,0.799816,0.841530,0.758101,0.804923,0.799816,0.841530,0.758101,0.804923,0.799816
std,0.038565,0.019471,0.017420,0.019046,0.019471,0.038565,0.017420,0.019046,0.019471,0.038565,0.017420,0.019046
min,0.674419,0.793939,0.765306,0.757928,0.793939,0.674419,0.765306,0.757928,0.793939,0.674419,0.765306,0.757928
25%,0.736434,0.830303,0.792517,0.785906,0.830303,0.736434,0.792517,0.785906,0.830303,0.736434,0.792517,0.785906
50%,0.759690,0.842424,0.809524,0.803488,0.842424,0.759690,0.809524,0.803488,0.842424,0.759690,0.809524,0.803488
75%,0.790698,0.854545,0.816327,0.810641,0.854545,0.790698,0.816327,0.810641,0.854545,0.790698,0.816327,0.810641
max,0.829457,0.890909,0.843537,0.841156,0.890909,0.829457,0.843537,0.841156,0.890909,0.829457,0.843537,0.841156
